TO DO:
- Implement differences between efficiency/power curve PSDs
- Update populated curve PSD to reflect Diameter units (Cell B3), Power (B4), Pwr/Eta cell (B5) 

##### Imports, File Setup

In [134]:
import os
import pandas as pd
import numpy as np
# import shutil
from openpyxl import load_workbook
# import re

In [59]:
# Dictionaries
curve_dict = {
    'Flow':'Q',
    'Head':'H',
    'Power':'P2',
    'Efficiency':'Eta2',
    'NPSH':'NPSH'
}

In [166]:
# Create class to hold each curve by product number/trim
class Curve:
    def __init__(self, pumpModel, dataframe):
        self.name = pumpModel
        self.pn = dataframe.iloc[0]['ProductNumber']
        self.df = dataframe
        self.frequency = dataframe.iloc[0]['Frequency']
        self.phase = dataframe.iloc[0]['Phase']
        self.trims = self.df['RPM(Curve nominal)'].unique().tolist()
        self.trim_curves = {}


    def __repr__(self) -> str:
        return f"Pump Model(name={self.name}, trim={self.trim}, dataframe=\n{self.df})"


    def return_xy_points(self, curveType:str):
        '''Returns 2 lists: 1 - flow values, 2 - Head or Power or Efficiency or NPSH'''
        x_values = self.df['Q'].tolist()
        y_values = self.df[curve_dict[curveType]].tolist()
        return x_values, y_values
    

    def create_grouped_trim_curves(self):
        '''Group df by the trim/speed column'''
        grouped = self.df.groupby('RPM(Curve nominal)')
        # Create mini dataframes for each trim within a pump model
        for group_trim, trim_df in grouped:
            self.trim_curves[group_trim] = trim_df[['Q','H','P2', 'P1',	'Eta2',	'Eta1',	'NPSH']]
        return
    

    # Define the function to calculate new values based on affinity laws
    # def apply_affinity_laws(row):
    #     N1, N2 = row['N1'], row['N2']
    #     Q2 = row['Q1'] * (N2 / N1)
    #     H2 = row['H1'] * (N2 / N1)**2
    #     P2 = row['P1'] * (N2 / N1)**3
    #     eta2 = np.nan
    #     eta1 = np.nan
    #     return pd.Series([Q2, H2, P2, eta2], index=['Q2', 'H2', 'P2', 'eta2'])



    def create_new_trim_df(self, n2):
        # Calculate new values for flow, head, power, npsh)
        flow_mult_exp = 1
        npsh_head_mult_exp = 2
        power_mult_exp = 3
        
        n1 = self.max_trim
        df = self.trim_curves[n1]

        
        df['Q2'] = df['Q'].apply(lambda x: x * ((n2/n1)**flow_mult_exp))
        print(f"original col: {df['Q']}")
        print(f"new col: {df['Q2']}")
        df['H'] = df['Q'].apply(lambda x: x * ((n2/n1)**npsh_head_mult_exp))
        df['NPSH'] = df['NPSH'].apply(lambda x: x * ((n2/n1)**npsh_head_mult_exp))
        df['P2'] = df['P2'].apply(lambda x: x * ((n2/n1)**power_mult_exp))
        df['P1'] = df['P1'].apply(lambda x: x * ((n2/n1)**power_mult_exp))
        df['Eta2'] = np.nan
        df['Eta1'] = np.nan

        # Add new dataframe to dictionary trim_curves
        self.trim_curves[n2] = df
        return


    @property
    def max_trim(self):
        return max(self.trims)

In [ ]:
# def process_data(action: str) -> None:
#     """
#     Processes data based on the specified action.

#     Parameters:
#     action (str): The action to perform. Available options are:
#         - 'load': Load the data.
#         - 'save': Save the data.
#         - 'delete': Delete the data.

#     Raises:
#     ValueError: If the action is not one of the available options.
#     """

#     available_actions = ['load', 'save', 'delete']

#     if action not in available_actions:
#         raise ValueError(f"Invalid action. Available options are: {', '.join(available_actions)}")

#     # Process the data based on the action
#     if action == 'load':
#         print("Loading data...")
#         # Add your loading logic here
#     elif action == 'save':
#         print("Saving data...")
#         # Add your saving logic here
#     elif action == 'delete':
#         print("Deleting data...")
#         # Add your deleting logic here

# # Example usage:
# process_data('load')


In [139]:
# This is the folder/file with the curve export csv
# myDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\1 - PROJECTS\SPE Integration\SPE_PumpCurve"
myDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\2 - AREAS\Alpha GXS Integration\Curve PSD\Archive"
myFile = "Pump Curves - ALPHA.csv"
filePath = os.path.join(myDir, myFile)

In [167]:
# This separates the GPI curve export by groups according to ProductNumber
# df= pd.read_csv(filePath, sep=";", index_col=False, skip_blank_lines=False) # FOR SPE
df= pd.read_csv(filePath, index_col=False, skip_blank_lines=False) # For ALPHA Testing
df = df.replace(',','.', regex=True)

# Drop NaN rows before grouping
df_non_nan = df.dropna(how='all')

# Forward fill productname and curve nominal columns for grouping 
df_non_nan['Product name'] = df_non_nan['Product name'].ffill()
df_non_nan['RPM(Curve nominal)'] = df_non_nan['RPM(Curve nominal)'].ffill()

# Group by the pump model column
grouped = df_non_nan.groupby('Product name')

# Make a model from each group 
group_objects = {}
for pumpModel, group_df in grouped:
    group_object = Curve(pumpModel=pumpModel, dataframe=group_df)
    group_object.create_grouped_trim_curves()
    group_objects[(f'{pumpModel}')] = group_object
    

C:\Users\104092\AppData\Local\Temp\ipykernel_69568\2295600585.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_nan['Product name'] = df_non_nan['Product name'].ffill()
C:\Users\104092\AppData\Local\Temp\ipykernel_69568\2295600585.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_nan['RPM(Curve nominal)'] = df_non_nan['RPM(Curve nominal)'].ffill()


In [168]:
# Iterate through list of models and new speeds that need to be created
n2 = 1800

for object_name in group_objects:
    model_object = group_objects[object_name]
    model_object.create_new_trim_df(n2)
    
    # Apply the function to each row
    # df[['Q2', 'H2', 'P2', 'eta2']] = df.apply(apply_affinity_laws, axis=1)


AttributeError: 'numpy.ndarray' object has no attribute 'apply'

In [ ]:
#TESTING 
# x_list, y_list = group_objects['ALPHA1 15-55F'].return_xy_points('Head')

group_objects['ALPHA1 15-55F'].trim_curves[3500]
# group_objects['ALPHA1 15-55F'].trim_curves[1800]

In [ ]:


# Leverage Luke's code to write these to xml for deployment

# Write to excel for data validation, or plot with matplotlib


# Testing

##### Functions

In [227]:
# def get_min_max_values(myList):

#     min_speed = myList[-1]
#     max_speed = myList[0]
#     max_nominal = max_speed

#     return min_speed, max_nominal, max_speed

In [228]:
# def findSpeedCells(my_df):
#     """ function to find which cell to start populating curve data based on RPM"""
#     # from openpyxl.utils.cell import coordinate_from_string, column_index_from_string, get_column_letter
#     from openpyxl.utils.cell import get_column_letter
#     speedCells = []
    
#     max_cols = 21 * len(my_df.RPM.unique())
#     row = 7    
#     first_col = 4
#     diameter_cols = list(range(first_col,max_cols,21))
    
#     for item in diameter_cols:
#         col = get_column_letter(item)
#         cell_coordinate = "{}{}".format(col,row)
#         speedCells.append(cell_coordinate)      

#     return(speedCells)

##### Main

In [229]:
def curveDataByPartNumber(data) -> dict:
    """Creates dictionary of dataframes for each PN """
    import math
    
    # This will become a dictionary with keys=pns and values=curve_data_dataframes
    dict_curveDataByPn = {}
    
    # Iterate through each row in pump curve data export 
    for index, value in data.iterrows():    
        
        # When a PN is encountered:
        # if not math.isnan(value['ProductNumber']):
        #     currentProductNumber = int(value['ProductNumber'])
        if type(value['ProductNumber']) == str:
            currentProductNumber = value['ProductNumber']
            
            # Resets list for each PN
            listOfFlows = []
            listOfHeads = []
            listOfPow = []
            listOfSpeeds = []
            listOfNPSH = []
            
        # Recording Q/H values to lists
        if value['RPM(Curve nominal)'] > 0:
            listOfFlows.append(value['Q'])
            listOfHeads.append(value['H'])
            listOfNPSH.append(value['NPSH'])
            listOfPow.append(value['P1'])
            listOfSpeeds.append(value['RPM(Curve nominal)'])
            
        # At end of Q/H values, store in dataframe before moving to next PN, or when end of data is reached
        if pd.isna(value['Q']) and pd.isna(value['H']) or (index == len(data)-1):    
            zipped = list(zip(listOfFlows, listOfHeads, listOfPow, listOfNPSH, listOfSpeeds))
            df = pd.DataFrame(zipped, columns=['Q','H','P1', 'NPSH','RPM'])

            # Drop rows that have NaNs, then add df to dictionary
            df = df.dropna()
            dict_curveDataByPn.update({currentProductNumber:df})

            continue

    return dict_curveDataByPn
    
# Creates dictionary with part numbers as keys, curves as dataframes for each key
curveDataDict = curveDataByPartNumber(data)    

In [230]:
def curveDataMatches(curveDataDict):  
    """ Create list of PNs that share curve data """

    curveDataCopy = curveDataDict.copy()
    listOfLists = []
    
    for eachPN, eachDF in curveDataDict.items():  
        listOfPNs = []
        for key, value in curveDataCopy.items():
            if value.equals(eachDF):
                listOfPNs.append(key)
        
        if listOfPNs not in listOfLists:
            listOfLists.append(listOfPNs)
    
    # print(f'list of lists: {listOfLists}')
    # print(f'list of pns: {listOfPNs}')
    
    return listOfLists
    
# Creates list of which part numbers share curve data
uniqueCurvePartnumbers = curveDataMatches(curveDataDict)

In [231]:
# def add_a_curve(list_unique_curves):  # WORKING. DO NOT MODIFY YET
#     """ CREATE new tabs for each unique curve family """
    
#     global wb
#     tab_to_copy = wb['NEW'] 
    
#     partnumbers = []
#     min_speeds = []
#     nom_speeds = []
#     max_speeds = []
    
#     # Add a tab for each unique curve
#     for item in list_unique_curves: 
        
#         list_of_speeds = []

#         # Add 1 tab
#         tabName = str(item[0])
#         # print(tabName)
#         wb.copy_worksheet(tab_to_copy).title = tabName

#         # Fill in all speeds/trims in column A
#         for key, value in curveDataDict[tabName].iterrows():
#             list_of_speeds.append(value['RPM'])
        
#         speed_set = sorted(set(list_of_speeds), reverse=True)
#         # print(tabName, speed_set)  
        
#         for index, eachSpeed in enumerate(speed_set):
#             cell_name = "{}{}".format('A', 10+index)
#             curveSheet = wb[tabName]
#             curveSheet[cell_name].value = int(eachSpeed)

#             min_rpm, nom_rpm, max_rpm = get_min_max_values(speed_set)
            
#             partnumbers.append(tabName)
#             min_speeds.append(min_rpm)
#             nom_speeds.append(nom_rpm)
#             max_speeds.append(max_rpm)
            
#             zipped = list(zip(partnumbers, min_speeds, nom_speeds, max_speeds))
#             df = pd.DataFrame(zipped, columns=['Partnumber','Min RPM','Nominal RPM', 'Max RPM'])
        
#     return df
    
# # Adds curve tabs for each unique curve
# # minmax_speeds_df = add_a_curve(uniqueCurvePartnumbers)

In [232]:
# Add 1 curve tab to workbook for every unique curve found (avoids duplicating data in SKB)
for item in uniqueCurvePartnumbers: 
    tabName = str(item[0])
    wb.copy_worksheet(wb['NEW']).title = tabName # Creates and renames blank PSD Tab as template for each new curve tab

In [ ]:
def extract_variable_speeds(df) -> list:
    ''' Takes curve data in dataframe format, and returns list of all unique speeds for variable speed pumps'''
    list_of_rpms = []

    for key, value in df.iterrows():
        list_of_rpms.append(value['RPM'])
    
    list_of_rpms = sorted(set(list_of_rpms), reverse=True)

    return list_of_rpms

# Iterate through list of unique curve names, and populate the corresponding workbook tab
for uniqueCurve in uniqueCurvePartnumbers:
    curveSheet = wb[uniqueCurve[0]]
    curve_data_df = curveDataDict[uniqueCurve[0]]
    
    # Fill Speed or trim in column A
    speed_set = extract_variable_speeds(curve_data_df)

    for index, eachSpeed in enumerate(speed_set):
        cell_name = "{}{}".format('A', 10+index)
        curveSheet[cell_name].value = int(eachSpeed)
        # print(f'For PN: {uniqueCurve[0]} -> cell: {cell_name} should have value: {int(eachSpeed)}. It currently has: {curveSheet[cell_name].value}')

    # Fill Curve, power/eff, NPSH data
    cellName = 'D7' 
    first_row_offset = 3

    for key, value in curve_data_df.iterrows():
        curveSheet[cellName].offset(first_row_offset + key, 0).value = value['Q']
        curveSheet[cellName].offset(first_row_offset + key, 1).value = value['H']
        curveSheet[cellName].offset(first_row_offset + key, 7).value = value['Q']
        # curveSheet[cellName].offset(first_row_offset + key, 8).value = value['Eta1']
        curveSheet[cellName].offset(first_row_offset + key, 8).value = value['P1']
        curveSheet[cellName].offset(first_row_offset + key,14).value = value['Q']
        curveSheet[cellName].offset(first_row_offset + key,15).value = value['NPSH']

    # Fill out corresponding row in Curve Header Data Tab
    # sheet = wb["Curve Header Data"]
    # sheet['B7'] = "SP"

    # cellName = 'A1' 
    # first_row_offset = 10

    # for index, speedData in curve_data_df.iterrows():
    #     print(f'Current pn: {uniqueCurve[0]}')
    #     print(f'{speedData}')


    #     sheet[cellName].offset((first_row_offset + index), 0).value = uniqueCurve[0]
    #     sheet[cellName].offset((first_row_offset + index), 2).value = speedData['RPM']
    #     sheet[cellName].offset((first_row_offset + index), 6).value = speedData['RPM']
    #     sheet[cellName].offset((first_row_offset + index), 7).value = speedData['RPM']
    #     sheet[cellName].offset((first_row_offset + index), 8).value = speedData['RPM']
    

In [234]:
# def speeds_for_curve_header_data():
    # # partnumbers = []
    # # min_speeds = []
    # # nom_speeds = []
    # # max_speeds = []

    # min_rpm, nom_rpm, max_rpm = get_min_max_values(speed_set)

    # partnumbers.append(tabName)
    # min_speeds.append(min_rpm)
    # nom_speeds.append(nom_rpm)
    # max_speeds.append(max_rpm)

    # zipped = list(zip(partnumbers, min_speeds, nom_speeds, max_speeds))
    # df = pd.DataFrame(zipped, columns=['Partnumber','Min RPM','Nominal RPM', 'Max RPM'])
#     return

In [235]:
# # Fill out Curve Header Data Tab
# newPumpFamilyName = "SP"
# sheet = wb["Curve Header Data"]
# sheet['B7'] = newPumpFamilyName

# cellName = 'A1' 
# first_row_offset = 10

# for index, speedData in df_of_speeds.iterrows():
#     sheet[cellName].offset((first_row_offset + index), 0).value = speedData['Partnumber']
#     sheet[cellName].offset((first_row_offset + index), 2).value = speedData['Nominal RPM']
#     sheet[cellName].offset((first_row_offset + index), 6).value = speedData['Nominal RPM']
#     sheet[cellName].offset((first_row_offset + index), 7).value = speedData['Min RPM']
#     sheet[cellName].offset((first_row_offset + index), 8).value = speedData['Max RPM']

In [236]:
def fillCurveHeaderDataTab(df_of_speeds):
    global wb
    newPumpFamilyName = "SP"
    sheet = wb["Curve Header Data"]

    cellName = 'A1' 
    first_row_offset = 10

    for index, speedData in df_of_speeds.iterrows():
        sheet[cellName].offset((first_row_offset + index), 0).value = speedData['Partnumber']
        sheet[cellName].offset((first_row_offset + index), 2).value = speedData['Nominal RPM']
        sheet[cellName].offset((first_row_offset + index), 6).value = speedData['Nominal RPM']
        sheet[cellName].offset((first_row_offset + index), 7).value = speedData['Min RPM']
        sheet[cellName].offset((first_row_offset + index), 8).value = speedData['Max RPM']

    sheet['B7'] = newPumpFamilyName
    wb.save(workingCopy)

    return

# fillCurveHeaderDataTab(minmax_speeds_df)

In [237]:
# Create list of PNs that share curve data, and inserts tab to illustrate
curveFamilySheet = wb.create_sheet("Shared Curves", 2)

for index, family in enumerate(uniqueCurvePartnumbers):
    row = index + 1  # Excel doesn't like 0-indexes

    # Fill 1st column in spreadsheet
    curveFamilySheet.cell(row=row, column=1).value = "Curve " + str(row)

    # Fill 2nd (or more) columns with partnumbers that share curves
    if len(family) == 1:
        curveFamilySheet.cell(row=row, column=2).value = family[0]
        # print('length of family is 1')
    else:
        curveFamilySheet.cell(row=row, column=2).value = family[0]
        curveFamilySheet.cell(row=row, column=3).value = family[1]
        # print(f'family[0]: {family[0]}, family[1]: {family[1]}')

In [238]:
# Save changes to excel sheet
wb.save(workingCopy)